In [1]:
import os
from scipy.io import loadmat
import pandas as pd
from GraphBuild import * 
import matplotlib.pyplot as plt
import random

In [2]:
mat_data = loadmat('chargerdata.mat') # Reading the charging station data

In [3]:
charger_data = mat_data['chargerdata']

In [4]:
Chargers_Select = []
# Loop through each EV charger entry
for charger_name in charger_data.dtype.names:
    daily_time = charger_data[charger_name][0,0]['daily_time'][0][0].flatten()
    daily_charging_power = charger_data[charger_name][0,0]['daily_charging_power'][0][0].flatten()
    daily_charging_current = charger_data[charger_name][0,0]['daily_charging_current'][0][0].flatten()
    daily_charging_voltage = charger_data[charger_name][0,0]['daily_charging_voltage'][0][0].flatten()
    latitude = charger_data[charger_name][0,0]['Latitude'][0][0].flatten()
    longitude = charger_data[charger_name][0,0]['Longitude'][0][0].flatten()
    data = {'Latitude':latitude, 
            'Longitude':longitude, 
            'daily_charging_current':daily_charging_current,
            'daily_charging_power':daily_charging_power*5,
            'daily_charging_voltage':daily_charging_voltage,
            'daily_time': daily_time}
    Chargers_Select.append({'name':charger_name, 'data':data})

In [5]:
for d in Chargers_Select:
    d['PBase'] = (max(d['data']['daily_charging_power'])*1.05)/1000 # slighlty more than maximum and converting it to kW

In [6]:
# Charging Stations
StationsInfo  =[{'no': 1, 'bus': 'l3104136','numphase':3, 'phaseconn':'.1.2.3', 'kV':12.47, 'indx':0, 'kw':Chargers_Select[0]['PBase'], 'kwh':Chargers_Select[0]['PBase']*10.5*3600, 'stored':0, 'reserve':0},
                {'no': 2, 'bus': 'l2895449','numphase':3, 'phaseconn':'.1.2.3', 'kV':12.47, 'indx':1, 'kw':Chargers_Select[1]['PBase'], 'kwh':Chargers_Select[1]['PBase']*10.5*3600, 'stored':0, 'reserve':0},
                {'no': 3, 'bus': 'l3010560','numphase':3, 'phaseconn':'.1.2.3', 'kV':12.47, 'indx':2, 'kw':Chargers_Select[2]['PBase'], 'kwh':Chargers_Select[2]['PBase']*10.5*3600, 'stored':0, 'reserve':0},
                {'no': 4, 'bus': 'l2876797','numphase':3, 'phaseconn':'.1.2.3', 'kV':12.47, 'indx':0, 'kw':Chargers_Select[0]['PBase'], 'kwh':Chargers_Select[0]['PBase']*10.5*3600, 'stored':0, 'reserve':0},
                {'no': 5, 'bus': 'l2876814','numphase':3, 'phaseconn':'.1.2.3', 'kV':12.47, 'indx':1, 'kw':Chargers_Select[1]['PBase'], 'kwh':Chargers_Select[1]['PBase']*10.5*3600, 'stored':0, 'reserve':0},
                {'no': 6, 'bus': 'l3081380','numphase':3, 'phaseconn':'.1.2.3', 'kV':12.47, 'indx':2, 'kw':Chargers_Select[2]['PBase'], 'kwh':Chargers_Select[2]['PBase']*10.5*3600, 'stored':0, 'reserve':0},
                {'no': 7, 'bus': 'l2766718','numphase':3, 'phaseconn':'.1.2.3', 'kV':12.47, 'indx':1, 'kw':Chargers_Select[1]['PBase'], 'kwh':Chargers_Select[1]['PBase']*10.5*3600, 'stored':0, 'reserve':0}]

In [7]:
StationsInfo 

[{'no': 1,
  'bus': 'l3104136',
  'numphase': 3,
  'phaseconn': '.1.2.3',
  'kV': 12.47,
  'indx': 0,
  'kw': 368.449361198086,
  'kwh': 13927385.853287652,
  'stored': 0,
  'reserve': 0},
 {'no': 2,
  'bus': 'l2895449',
  'numphase': 3,
  'phaseconn': '.1.2.3',
  'kV': 12.47,
  'indx': 1,
  'kw': 261.20449606385563,
  'kwh': 9873529.951213742,
  'stored': 0,
  'reserve': 0},
 {'no': 3,
  'bus': 'l3010560',
  'numphase': 3,
  'phaseconn': '.1.2.3',
  'kV': 12.47,
  'indx': 2,
  'kw': 544.6198183458613,
  'kwh': 20586629.133473556,
  'stored': 0,
  'reserve': 0},
 {'no': 4,
  'bus': 'l2876797',
  'numphase': 3,
  'phaseconn': '.1.2.3',
  'kV': 12.47,
  'indx': 0,
  'kw': 368.449361198086,
  'kwh': 13927385.853287652,
  'stored': 0,
  'reserve': 0},
 {'no': 5,
  'bus': 'l2876814',
  'numphase': 3,
  'phaseconn': '.1.2.3',
  'kV': 12.47,
  'indx': 1,
  'kw': 261.20449606385563,
  'kwh': 9873529.951213742,
  'stored': 0,
  'reserve': 0},
 {'no': 6,
  'bus': 'l3081380',
  'numphase': 3,
  '

In [7]:
# Initialize Circuit
FolderName = os.path.dirname(os.path.realpath("__file__"))
DSSfile = r""+ FolderName+ "\Master.dss"
Ckt_obj = CircuitSetup(DSSfile, StationsInfo)  #creating a DSS object instance

In [8]:
G_original =  build_graph(Ckt_obj)
nx.readwrite.gml.write_gml(G_original,"8500busEx.gml") #Graph undirected with edge features and node features which are constant

In [9]:
node_list=list(G_original.nodes())
edge_list=list(G_original.edges())

In [10]:
input_file = 'LoadShape1.xlsx'
sheet_name = 'LoadShape1'  # Change to the sheet you want to split
df = pd.read_excel(input_file, sheet_name=sheet_name)

points_per_day = 24  # Assuming 1-hour resolution (24 points = 24 hours)
# Convert to flat list
loadshape_values = df.iloc[:, 0].values.tolist()
# Check total number of complete days available
num_days = len(loadshape_values) // points_per_day
# Slice the loadshape into daily
LoadShapes = []
for i in range(num_days):
    start_idx = i * points_per_day
    end_idx = (i + 1) * points_per_day
    daily_shape = loadshape_values[start_idx:end_idx]
    LoadShapes.append(daily_shape)

In [11]:
# Initializing Load Demand shape
Ckt_obj.dss.Text.Command(f"New LoadShape.LoadVar")

In [12]:
# Initializing Storage Shapes
for i in range(len(StationsInfo)):
    shape_name = "StorageShape"+str(StationsInfo[i]['no'])
    Ckt_obj.dss.Text.Command(f"New LoadShape.{shape_name}")  

In [13]:
loadshape_day = random.choice(LoadShapes)

In [14]:
charging_profiles = []
for i in range(len(StationsInfo)):
    sindx = StationsInfo[i]['indx']
    charge_P = -1*(Chargers_Select[sindx]['data']['daily_charging_power']/(StationsInfo[i]['kw']*1000)) # converting the charging power to kW, normalizing wrt to max Power and -ve (indicate charging)
    charging_profiles.append(charge_P)

In [15]:
def Powerflow_Timeseries(Ckt_obj, loadshape_day, charging_profiles):
    # Edit the LoadShape in opendss
    Ckt_obj.dss.Text.Command(f"Edit LoadShape.LoadVar npts={len(loadshape_day)} interval=1 mult=(" + ' '.join(map(str, loadshape_day)) + ")")
    # Assign Load shapes
    Ckt_obj.dss.Text.Command("BatchEdit Load..* daily=LoadVar")
    
    # Edit the Storage Shape in opendss
    for i in range(len(StationsInfo)):
        shape_name = "StorageShape"+str(StationsInfo[i]['no'])
        charge_P = charging_profiles[i]
        Ckt_obj.dss.Text.Command(f'Edit LoadShape.{shape_name} npts={len(charge_P)} pmult=({ " ".join(map(str, charge_P)) }) sinterval=1')
        # Assign Storage shapes
        elem_name = 'ChargeStatn'+str(StationsInfo[i]['no'])
        Ckt_obj.dss.Text.Command(f'Edit Storage.{elem_name} Daily={shape_name}')
          
    # Time-series simulation        
    V_node_Sc = {bus: [] for bus in node_list}
    flow_branch_Sc= {br: [] for br in edge_list}
    powers_Sc = {stations['no']:[] for stations in StationsInfo}
    states_Sc = {stations['no']:[] for stations in StationsInfo}
    
    Ckt_obj.dss.Text.Command("Set mode=daily")
    Ckt_obj.dss.Text.Command("Set stepsize=5m")
    Ckt_obj.dss.Text.Command("Set number=1")
    t= 0
    while t<24:
            Ckt_obj.dss.Solution.Solve()
            # Get Node Voltages
            for bus in node_list:
                V=Bus(Ckt_obj,bus).Vmag
                V_node_Sc[bus].append(V) 
                
            # Get Storage powers  
            i = Ckt_obj.dss.Storages.First()
            while i>0:
                    sno = StationsInfo[i-1]['no']
                    powers_Sc[sno].append(sum(Ckt_obj.dss.CktElement.Powers()[::2])) # Circuit element power 
                    states_Sc[sno].append(Ckt_obj.dss.Storages.State()) # Storage state 
                    i= Ckt_obj.dss.Storages.Next()       
                    
            # Get branchflows  
            for (u,v) in edge_list:
                branch_label = G_original[u][v]['Label']
                branch_device = G_original[u][v]['Device']
                branch_elem = f"{branch_device}.{branch_label}"
                branch_pflow = Branch(Ckt_obj,branch_elem).flow
                branchflow = np.sum(branch_pflow)
                flow_branch_Sc[(u,v)].append(branchflow)
                    
            t = Ckt_obj.dss.Solution.DblHour()
        
    return  V_node_Sc, powers_Sc, flow_branch_Sc, states_Sc
    

In [16]:
V_node_Sc, powers_Sc, flow_branch_Sc, states_Sc = Powerflow_Timeseries(Ckt_obj, loadshape_day, charging_profiles)